In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [7]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (shared):
                      backbone has      10896 params (~  0.2) %
                    R_backbone has    3745798 params (~ 67.7) %
                    t_backbone has    1774339 params (~ 32.1) %
total params:              5531033
cam2cam estimation => adding 5531033 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-20:02:53
#[do_train]: epoch    0 has started!
tensor([[-0.6964,  0.0729, -0.7140],
        [-0.1114,  0.9718,  0.2078],
        [ 0.7090,  0.2242, -0.6686]], device='cuda:0')
tensor([[-0.6964,  0.0729, -0.7140],
        [-0.1114,  0.9718,  0.2078],
        [ 0.7090,  0.2242, -0.6686]], device='cuda:0')
tensor([[[-0.0294, -0.1400,  0.9897],
         [ 0.1167,  0.9829,  0.1425],
         [-0.9927,  0.1197, -0.0126]]], device='cuda:0')


ZeroDivisionError: division by zero